In [237]:
import numpy as np
import re
import xlrd

In [ ]:
class TalentsInfo:
    def __init__(self, excel_path='指挥喵/天赋.xlsx'):
        

In [238]:
doc = xlrd.open_workbook('指挥喵/天赋.xlsx')

In [278]:
sheet = doc.sheet_by_index(0)
print(sheet.name, sheet.ncols, sheet.nrows)

Sheet1 18 56


In [279]:
data = []
for i in range(sheet.nrows):
    row = []
    for j in range(sheet.ncols):
        row.append(get_cell(i, j))
    data.append(row)

In [280]:
def parsed(value):
    if isinstance(value, str):
        return value.strip()
    return value
    
def get_cell(i, j):
    for x0, x1, y0, y1 in sheet.merged_cells:
        if x0 == i and y0 == j:
            break
        if x0 <= i < x1 and y0 <= j < y1:
            return parsed(sheet.cell(x0, y0).value)
    return parsed(sheet.cell(i, j).value)


In [281]:
names = data[2]
split_idx = [4, 16]

a, b = split_idx
desc = names[:a]
score = names[a:b]
chance = names[b:]


In [282]:
info = {}
for line in data[3:]:
    if not any(line):
        continue
    item = {
        'name': line[0],
        'desc': dict(zip(names, line[:a])),
        'scores': dict(zip(score, line[a:b])),
        'chance': dict(zip(chance, line[b:])),
    }
    name = item['name']
    if name not in info:
        info[name] = item
    else:
        info[name]['desc'].append(item['desc'])
        for key in score:
            if item['scores'][key]:
                info[name]['scores'][key] = item['scores'][key]
    if name == '':
        print(line)
        
for item in info.values():
    for key in score:
        if item['scores'][key] == '':
            del item['scores'][key]

In [283]:
info

{'一发入魂': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '一发入魂',
   '描述': '战巡、战列炮击提高15点，主炮暴击率提高3%',
   '特殊天赋': 1.0,
   '简称': '发'},
  'name': '一发入魂',
  'scores': {'战列': 9.0, '战巡': 9.0}},
 '不动如山': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '不动如山', '描述': '舰队成员受到伤害降低3%', '特殊天赋': 1.0, '简称': '山'},
  'name': '不动如山',
  'scores': {'战列': 6.0,
   '战巡': 6.0,
   '正航': 6.0,
   '潜艇': 2.0,
   '炮巡': 6.0,
   '维修': 6.0,
   '航战': 6.0,
   '轻航': 6.0,
   '轻雷巡': 6.0,
   '重炮': 6.0,
   '重雷巡': 6.0,
   '驱逐': 6.0}},
 '侵略如火': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '侵略如火', '描述': '舰队成员造成伤害提高3%', '特殊天赋': 1.0, '简称': '火'},
  'name': '侵略如火',
  'scores': {'战列': 10.0,
   '战巡': 10.0,
   '正航': 10.0,
   '潜艇': 10.0,
   '炮巡': 10.0,
   '航战': 10.0,
   '轻航': 10.0,
   '轻雷巡': 10.0,
   '重炮': 10.0,
   '重雷巡': 10.0,
   '驱逐': 10.0}},
 '其徐如林': {'chance': {'SR': 1.0, 'SSR': 1.0},
  'desc': {'天赋名': '其徐如林',
   '描述': '舰队防空、反潜提高15点，命中、机动提高3点',
   '特殊天赋': 1.0,
   '简称': '林'},
  'name': '其徐如林',
  'scores': {'战列': 6

In [245]:
def sort_talents(fleet, talents=info):
    def key(item):
        item['ScoreInfo'] = []
        score = 0
        for key in fleet:
            score_add = item['scores'].get(key,0) * fleet[key]
            if score_add:
                item['ScoreInfo'].append((key, fleet[key], item['scores'][key],))
                score += score_add
        item['Score'] = score
        return -score
    
    ship_count = sum(fleet.values())
    items = list(talents.values())
    items.sort(key=key)
    items = [{
        'score': item['Score'], 
        'name': item['name'], 
        'scoreInfo': item['ScoreInfo']} 
        for item in items if item['Score'] > 0]
    return items

def show_scores(scores):
    for i, item in enumerate(scores):
        print('{1:2d}: {0[score]:5.3f}, {0[name]:}  {0[scoreInfo]}'.format(item, i+1))
    print()
        
        
def show_talents(fleet, talents=info, limit=None):
    print(fleet)
    items = sort_talents(fleet, talents)
    if limit:
        items = items[:limit]
    show_scores(items)

In [246]:
short_dict = {
    '航': {'正航', '轻航'},
    '战': {'战列', '战巡', '航战'},
    '驱': {'驱逐'},
    '巡': {'轻雷巡', '重雷巡', '炮巡'},
    '潜': {'潜艇'}
}

def search_talent(ship_types, talent_short):
    candidates = []
    for talent in info.values():
        if talent_short == talent['desc']['简称']:
            candidates.append(talent)
    if len(candidates) == 1:
        return candidates[0]['name']
    for talent in candidates:
        for ship_type in ship_types:
            if talent['scores'].get(ship_type, 0) > 0:
                return talent['name']
    return {"Error": {'ship_types': ship_types, 'talent_short': talent_short}, 'candidates': candidates}
    
def get_talents(cat):
    if isinstance(cat, str):
        cat = re.search("(?i)^(.)([ac ])(.+)$", cat).groups()
    s_type, s_cmd, s_talents = cat
    ship_types = short_dict[s_type]
    
    talents = []
    for s in s_talents:
        talents.append(search_talent(ship_types, s))
    return talents

In [247]:
def cat_score(fleet, cat, show_scores=False):
    talents = {key: info[key] for key in get_talents(cat)}
    scores = sort_talents(fleet, talents)
    total_score = sum([item['score'] for item in scores])
    if show_scores:
        print('{:5.3f}: {} for {}'.format(total_score, cat, fleet, ))
        show_scores(scores)
    return {"Score": total_score, "Info": scores}

In [248]:
from itertools import combinations_with_replacement

In [249]:
fleet_main = ['战列', '正航', '轻航', '重炮', '维修']
fleet_scout = ['驱逐', '轻雷巡', '重雷巡', '炮巡']
fleets_extra = [{"潜艇": i}for i in [1,2,3]]
def all_fleets(choices, include=None):
    results = []
    for count in [1,2,3]:
        for choice in combinations_with_replacement(choices, count):
            tmp = {}
            for item in set(choice):
                tmp[item] = choice.count(item)
            if include is None or include(tmp):
                results.append(tmp)
    return results

def filt_all(fleet):
    if fleet.get('维修', 0) > 1:
        return False
    if fleet.get('重炮', 0) > 2:
        return False
    
    if fleet.get('重雷巡', 0) > 2:
        return False
    return True

fleets = fleets_extra
for main in all_fleets(fleet_main, filt_all):
    for scout in all_fleets(fleet_scout, filt_all):
        fleets.append(dict(**main, **scout))
print('Get %d fleets' % len(fleets))

Get 1587 fleets


In [250]:
for ship in fleet_main + fleet_scout + ['潜艇']:
    show_talents({ship: 1}, limit=10)

{'战列': 1}
 1: 10.000, 侵略如火  [('战列', 1, 10.0)]
 2: 9.000, 一发入魂  [('战列', 1, 9.0)]
 3: 8.000, 既定的命运  [('战列', 1, 8.0)]
 4: 8.000, 炮击新手·主力  [('战列', 1, 8.0)]
 5: 6.000, 其徐如林  [('战列', 1, 6.0)]
 6: 6.000, 不动如山  [('战列', 1, 6.0)]
 7: 6.000, 装填新手·战列  [('战列', 1, 6.0)]
 8: 6.000, 新手观测士·主力  [('战列', 1, 6.0)]
 9: 5.000, 小小的奇迹  [('战列', 1, 5.0)]
10: 5.000, 新晋指挥官·战列  [('战列', 1, 5.0)]

{'正航': 1}
 1: 10.000, 侵略如火  [('正航', 1, 10.0)]
 2: 8.000, 新晋指挥官·空母  [('正航', 1, 8.0)]
 3: 7.000, 王牌机师  [('正航', 1, 7.0)]
 4: 7.000, 航空新兵·空母  [('正航', 1, 7.0)]
 5: 6.000, 其徐如林  [('正航', 1, 6.0)]
 6: 6.000, 不动如山  [('正航', 1, 6.0)]
 7: 6.000, 既定的命运  [('正航', 1, 6.0)]
 8: 6.000, 新晋指挥官·白鹰  [('正航', 1, 6.0)]
 9: 4.000, 新手整备士  [('正航', 1, 4.0)]
10: 4.000, 新晋指挥官·重樱  [('正航', 1, 4.0)]

{'轻航': 1}
 1: 10.000, 侵略如火  [('轻航', 1, 10.0)]
 2: 7.000, 航空新兵·空母  [('轻航', 1, 7.0)]
 3: 6.000, 其徐如林  [('轻航', 1, 6.0)]
 4: 6.000, 不动如山  [('轻航', 1, 6.0)]
 5: 6.000, 既定的命运  [('轻航', 1, 6.0)]
 6: 6.000, 新晋指挥官·白鹰  [('轻航', 1, 6.0)]
 7: 4.000, 新手整备士  [('轻航', 1, 4.0)]
 8

# 指挥喵列表

```
航a航樱

战a指空山
战a炮皇
驱c指耐装

潜a中指
潜a指中

驱a运机

巡a炮皇

航a樱指潜
航a耐中指
航a装机航
航a指耐装

战a装耐见
战a指耐空

驱c耐装炮中
驱c指炮中
驱c雷机指

潜a装

战a耐空
战a炮装
战a中空
战a炮
战a机中

驱A指炮
驱a潜耐

航a机指
航A航装
航a指航
航a指潜
航a装中
航a潜耐

驱A指耐
巡a雷指

巡a雷炮

航a航鹰
航A指

驱C耐指
驱c装雷

```

In [257]:
with open('指挥喵/cats.md', 'r', -1, 'UTF8') as fl:
    content = fl.read()
ship_abbr = ''.join(short_dict.keys())
talent_abbr = {talent['desc']['简称'] for talent in info.values()}
talent_abbr.discard('-')
talent_abbr = ''.join(talent_abbr)

cats = set(re.findall(r'(?mi)([%s][ac ][%s]+)' % (ship_abbr, talent_abbr), content))
print(cats)
for cat in cats.copy():
    if cat[1] == ' ':
        for cmd in 'acAC':
            if (cat[0], cmd, cat[2]) in cats:
                cats.discard(cat)
cats = list(cats)
cats.sort()
print("Get %d cats" % len(cats))
print('\n'.join([''.join(cat) for cat in cats]))

{'驱a雷炮', '战a机中', '驱A指炮', '巡a雷炮', '驱c耐装炮中', '巡a炮皇', '巡a雷指', '战a指空山', '航a航鹰', '潜a中指', '航a指航', '航a装中', '航a樱指潜', '驱C耐指', '战a耐空', '航A航装', '航a航樱', '战a装耐见', '驱A指耐', '战a炮装', '航a机指', '驱a运机', '潜a装', '航A指', '驱c指耐装', '航a装机航', '战a指耐空', '航a指耐装', '驱c装雷', '潜a指中', '航a指潜', '航a潜耐', '驱a潜耐', '驱c指炮中', '战a炮皇', '战a中空', '驱c雷机指', '战a炮', '航a耐中指'}
Get 39 cats
巡a炮皇
巡a雷指
巡a雷炮
战a中空
战a指空山
战a指耐空
战a机中
战a炮
战a炮皇
战a炮装
战a耐空
战a装耐见
潜a中指
潜a指中
潜a装
航A指
航A航装
航a指潜
航a指耐装
航a指航
航a机指
航a樱指潜
航a潜耐
航a耐中指
航a航樱
航a航鹰
航a装中
航a装机航
驱A指炮
驱A指耐
驱C耐指
驱a潜耐
驱a运机
驱a雷炮
驱c指炮中
驱c指耐装
驱c耐装炮中
驱c装雷
驱c雷机指


In [262]:
def get_tuple_info(item):
    if isinstance(item, dict):
        return tuple((key, get_tuple_info(value)) for key, value in item.items())
    if isinstance(item, list):
        return tuple((get_tuple_info(value) for value in item))
    return item
cats.append("驱 火魂指雷")

cat_best = []
for i, cat in enumerate(cats):
    all_info = []
    for j, fleet in enumerate(fleets):
        score = cat_score(cat=cat, fleet=fleet)
        score['Fleet'] = {}
#         print(score)
        for scoreinfo in score['Info']:
#             print(scoreinfo)
            for item in scoreinfo['scoreInfo']:
                score['Fleet'][item[0]] = item[1]
        all_info.append(get_tuple_info(score))
    all_info = list(set(all_info))
    all_info.sort(key=lambda a: a[0][1], reverse=True)
    cat_best.append((cat, all_info[:5]))

In [271]:
talents = get_talents(cat_best[0][0])
cost = [info[key]['desc']['特殊天赋'] or 3 for key in talents]
print(talents, sum(cost))

['侵略如火', '水雷魂', '新晋指挥官·驱逐', '新人雷击士·驱逐'] 8.0


In [293]:
def get_cat_cost(cat):
    talents = get_talents(cat[0])
    cost = sum([info[key]['desc']['特殊天赋'] or 3 for key in talents])
    return cost

def get_cat_score(cat):
    return cat[1][0][0][1] / get_cat_cost(cat)


cat_best.sort(key=get_cat_score, reverse=True)
for cat in cat_best[:16]:
    print('Cost', get_cat_cost(cat), 'Score', get_cat_score(cat))
    display(cat)
    print()

Cost 8.0 Score 16.5


('驱 火魂指雷',
 [(('Score', 132.0),
   ('Info',
    ((('score', 60.0),
      ('name', '侵略如火'),
      ('scoreInfo', (('轻航', 1, 10.0), ('正航', 2, 10.0), ('驱逐', 3, 10.0)))),
     (('score', 27.0), ('name', '水雷魂'), ('scoreInfo', (('驱逐', 3, 9.0),))),
     (('score', 24.0), ('name', '新晋指挥官·驱逐'), ('scoreInfo', (('驱逐', 3, 8.0),))),
     (('score', 21.0),
      ('name', '新人雷击士·驱逐'),
      ('scoreInfo', (('驱逐', 3, 7.0),))))),
   ('Fleet', (('轻航', 1), ('正航', 2), ('驱逐', 3)))),
  (('Score', 132.0),
   ('Info',
    ((('score', 60.0),
      ('name', '侵略如火'),
      ('scoreInfo', (('轻航', 3, 10.0), ('驱逐', 3, 10.0)))),
     (('score', 27.0), ('name', '水雷魂'), ('scoreInfo', (('驱逐', 3, 9.0),))),
     (('score', 24.0), ('name', '新晋指挥官·驱逐'), ('scoreInfo', (('驱逐', 3, 8.0),))),
     (('score', 21.0),
      ('name', '新人雷击士·驱逐'),
      ('scoreInfo', (('驱逐', 3, 7.0),))))),
   ('Fleet', (('轻航', 3), ('驱逐', 3)))),
  (('Score', 132.0),
   ('Info',
    ((('score', 60.0),
      ('name', '侵略如火'),
      ('scoreInfo',
       ((


Cost 4.0 Score 12.0


('驱a运机',
 [(('Score', 48.0),
   ('Info',
    ((('score', 48.0),
      ('name', '既定的命运'),
      ('scoreInfo',
       (('重炮', 1, 8.0), ('战列', 2, 8.0), ('轻雷巡', 1, 8.0), ('重雷巡', 2, 8.0)))),)),
   ('Fleet', (('重炮', 1), ('战列', 2), ('轻雷巡', 1), ('重雷巡', 2)))),
  (('Score', 48.0),
   ('Info',
    ((('score', 47.0),
      ('name', '既定的命运'),
      ('scoreInfo',
       (('重炮', 2, 8.0), ('战列', 1, 8.0), ('重雷巡', 2, 8.0), ('驱逐', 1, 7.0)))),
     (('score', 1.0), ('name', '操舵手·小型舰'), ('scoreInfo', (('驱逐', 1, 1.0),))))),
   ('Fleet', (('重炮', 2), ('战列', 1), ('重雷巡', 2), ('驱逐', 1)))),
  (('Score', 48.0),
   ('Info',
    ((('score', 46.0),
      ('name', '既定的命运'),
      ('scoreInfo',
       (('重炮', 1, 8.0), ('战列', 2, 8.0), ('炮巡', 1, 8.0), ('驱逐', 2, 7.0)))),
     (('score', 2.0), ('name', '操舵手·小型舰'), ('scoreInfo', (('驱逐', 2, 1.0),))))),
   ('Fleet', (('重炮', 1), ('战列', 2), ('炮巡', 1), ('驱逐', 2)))),
  (('Score', 48.0),
   ('Info',
    ((('score', 46.0),
      ('name', '既定的命运'),
      ('scoreInfo', (('战列', 3, 8.0


Cost 7.0 Score 8.142857142857142


...

('战a装耐见',
 [(('Score', 48.0),
   ('Info',
    ((('score', 24.0), ('name', '见敌必战'), ('scoreInfo', (('轻雷巡', 3, 8.0),))),
     (('score', 18.0), ('name', '装填新手·战列'), ('scoreInfo', (('战列', 3, 6.0),))),
     (('score', 6.0), ('name', '轮机手·战列'), ('scoreInfo', (('战列', 3, 2.0),))))),
   ('Fleet', (('轻雷巡', 3), ('战列', 3)))),
  (('Score', 48.0),
   ('Info',
    ((('score', 24.0),
      ('name', '见敌必战'),
      ('scoreInfo', (('轻雷巡', 1, 8.0), ('重雷巡', 2, 8.0)))),
     (('score', 18.0), ('name', '装填新手·战列'), ('scoreInfo', (('战列', 3, 6.0),))),
     (('score', 6.0), ('name', '轮机手·战列'), ('scoreInfo', (('战列', 3, 2.0),))))),
   ('Fleet', (('轻雷巡', 1), ('重雷巡', 2), ('战列', 3)))),
  (('Score', 48.0),
   ('Info',
    ((('score', 24.0),
      ('name', '见敌必战'),
      ('scoreInfo', (('轻雷巡', 2, 8.0), ('重雷巡', 1, 8.0)))),
     (('score', 18.0), ('name', '装填新手·战列'), ('scoreInfo', (('战列', 3, 6.0),))),
     (('score', 6.0), ('name', '轮机手·战列'), ('scoreInfo', (('战列', 3, 2.0),))))),
   ('Fleet', (('轻雷巡', 2), ('重雷巡', 1), ('战


Cost 9 Score 6.666666666666667


('驱c指耐装',
 [(('Score', 60.0),
   ('Info',
    ((('score', 24.0), ('name', '新晋指挥官·驱逐'), ('scoreInfo', (('驱逐', 3, 8.0),))),
     (('score', 21.0), ('name', '轮机手·驱逐'), ('scoreInfo', (('驱逐', 3, 7.0),))),
     (('score', 15.0),
      ('name', '装填新手·驱逐'),
      ('scoreInfo', (('驱逐', 3, 5.0),))))),
   ('Fleet', (('驱逐', 3),))),
  (('Score', 40.0),
   ('Info',
    ((('score', 16.0), ('name', '新晋指挥官·驱逐'), ('scoreInfo', (('驱逐', 2, 8.0),))),
     (('score', 14.0), ('name', '轮机手·驱逐'), ('scoreInfo', (('驱逐', 2, 7.0),))),
     (('score', 10.0),
      ('name', '装填新手·驱逐'),
      ('scoreInfo', (('驱逐', 2, 5.0),))))),
   ('Fleet', (('驱逐', 2),))),
  (('Score', 20.0),
   ('Info',
    ((('score', 8.0), ('name', '新晋指挥官·驱逐'), ('scoreInfo', (('驱逐', 1, 8.0),))),
     (('score', 7.0), ('name', '轮机手·驱逐'), ('scoreInfo', (('驱逐', 1, 7.0),))),
     (('score', 5.0), ('name', '装填新手·驱逐'), ('scoreInfo', (('驱逐', 1, 5.0),))))),
   ('Fleet', (('驱逐', 1),))),
  (('Score', 0), ('Info', ()), ('Fleet', ()))])


Cost 6 Score 6.5


('巡a雷指',
 [(('Score', 39.0),
   ('Info',
    ((('score', 21.0),
      ('name', '新晋指挥官·巡洋'),
      ('scoreInfo', (('轻雷巡', 1, 7.0), ('重雷巡', 2, 7.0)))),
     (('score', 18.0),
      ('name', '新人雷击士·巡洋'),
      ('scoreInfo', (('轻雷巡', 1, 6.0), ('重雷巡', 2, 6.0)))))),
   ('Fleet', (('轻雷巡', 1), ('重雷巡', 2)))),
  (('Score', 39.0),
   ('Info',
    ((('score', 21.0),
      ('name', '新晋指挥官·巡洋'),
      ('scoreInfo', (('轻雷巡', 3, 7.0),))),
     (('score', 18.0),
      ('name', '新人雷击士·巡洋'),
      ('scoreInfo', (('轻雷巡', 3, 6.0),))))),
   ('Fleet', (('轻雷巡', 3),))),
  (('Score', 39.0),
   ('Info',
    ((('score', 21.0),
      ('name', '新晋指挥官·巡洋'),
      ('scoreInfo', (('轻雷巡', 2, 7.0), ('重雷巡', 1, 7.0)))),
     (('score', 18.0),
      ('name', '新人雷击士·巡洋'),
      ('scoreInfo', (('轻雷巡', 2, 6.0), ('重雷巡', 1, 6.0)))))),
   ('Fleet', (('轻雷巡', 2), ('重雷巡', 1)))),
  (('Score', 33.0),
   ('Info',
    ((('score', 21.0),
      ('name', '新晋指挥官·巡洋'),
      ('scoreInfo', (('轻雷巡', 2, 7.0), ('炮巡', 1, 7.0)))),
     (('score',